In [ ]:
#visualization tool for displaying long load/processing times
!pip install tqdm
#data processing
!pip install pandas
#logging output
!pip install loguru==0.7.0
#convenient pretty printing library
!pip install rich
#data framework for LLM applications
!pip install llama-index==0.9.29

In [ ]:
!pip install pandas

In [ ]:
#openai Tokenizer library
!pip install tiktoken

In [ ]:
#workhorse for converting text into embeddings/vectors
!pip install sentence-transformers==2.2.2

In [ ]:
import os
import json
import pandas as pd
from typing import List, Union
import tiktoken



In [25]:
def load_json(file_path: str) -> list[dict]:
        '''
        Loads json file from disk.
        '''
        with open(file_path) as f:
            data = json.load(f)
        return data

In [8]:
res=load_json('../data/dataset.json')

In [9]:
print(res[0]['content'])

Diagnostic Tests for Diabetes: There is insufficient evidence to support the use of continuous glucose monitoring for screening or diagnosing prediabetes or diabetes. In the absence of unequivocal hyperglycemia (e.g., hyperglycemic crisis), diagnosis of type 2 diabetes requires confirmatory testing, which can be a different test on the same day or the same test on a different day. Marked discordance between A1C and repeated blood glucose measurements should raise the possibility of a problem or interference with either test. Classification: Classification of diabetes type is not always straightforward at presentation, and misdiagnosis is common. Section 2: Diagnosis and Classification of Diabetes: Gestational diabetes mellitus (GDM; detected at 24–28 weeks of gestation in individuals without previously identified diabetes or high-risk glucose metabolism), Diabetes from other causes (e.g., monogenic diabetes syndromes, diseases of the exocrine pancreas, and drug- or chemical-induced dia

In [10]:
contents_list=[item['content']for item in res]
print(len(contents_list))

14


In [11]:
contents_count=[len(content.split()) for content in contents_list]
print(contents_count)

[738, 861, 958, 950, 382, 772, 842, 452, 563, 464, 771, 873, 701, 908]


In [26]:
import pandas as pd
df = pd.DataFrame(contents_count, columns=['# Words'])
df.describe()

,# Words
count,14.000000
mean,731.071429
std,192.870807
min,382.000000
25%,597.500000
50%,771.500000
75%,870.000000
max,958.000000


In [45]:
import tiktoken # bad ass tokenizer library for use with OpenAI LLMs

#instantiate tokenizer for use with ChatGPT-3.5-Turbo
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo-0125')

ModuleNotFoundError: No module named 'tiktoken'

In [ ]:
tokens = encoding.encode_batch(contents_list)
token_counts = list(map(len, tokens))
token_df = pd.DataFrame(token_counts, columns=['# Tokens'])
token_df.describe()

,# Tokens
count,14.000000
mean,1096.214286
std,307.034245
min,560.000000
25%,835.000000
50%,1201.000000
75%,1355.000000
max,1433.000000


In [ ]:
import numpy as np
token__to_word_ratio=round(np.mean(token_counts)/np.mean(contents_count), 2)

In [ ]:
print(token__to_word_ratio)

1.5


In [ ]:
from llama_index.text_splitter import SentenceSplitter #one of the best on the market

#set chunk size and instantiate your SentenceSplitter
chunk_size = 256
gpt35_txt_splitter = SentenceSplitter(chunk_size=chunk_size, tokenizer=encoding.encode, chunk_overlap=0)

In [ ]:
def split_contents(corpus: list[dict],
                   text_splitter: SentenceSplitter,
                   content_field: str='content'
                   ) -> list[list[str]]:
    '''
    Given a corpus of "documents" with text content, this function splits the
    content field into chunks sizes as specified by the text_splitter.

    Example
    -------
    corpus = [
            {'title': 'This is a cool show', 'content': 'There is so much good content on this show. \
              This would normally be a really long block of content. ... But for this example it will not be.'},
            {'title': 'Another Great Show', 'content': 'The content here is really good as well.  If you are \
              reading this you have too much time on your hands. ... More content, blah, blah.'}
           ]

    output = split_contents(data, text_splitter, content_field="content")

    output >>> [['There is so much good content on this show.', 'This would normally be a really long block of content.', \
                 'But for this example it will not be'],
                ['The content here is really good as well.', 'If you are reading this you have too much time on your hands.', \
                 'More content, blah, blah.']
                ]
    '''

    ########################
    # START YOUR CODE HERE #
    ########################
    return [text_splitter.split_text(doc[content_field]) for doc in corpus]


In [ ]:
opt = split_contents(res, gpt35_txt_splitter, content_field='content')
print(opt[0])
for o in opt[0]:
  print(o)

['Diagnostic Tests for Diabetes: There is insufficient evidence to support the use of continuous glucose monitoring for screening or diagnosing prediabetes or diabetes. In the absence of unequivocal hyperglycemia (e.g., hyperglycemic crisis), diagnosis of type 2 diabetes requires confirmatory testing, which can be a different test on the same day or the same test on a different day. Marked discordance between A1C and repeated blood glucose measurements should raise the possibility of a problem or interference with either test. Classification: Classification of diabetes type is not always straightforward at presentation, and misdiagnosis is common. Section 2: Diagnosis and Classification of Diabetes: Gestational diabetes mellitus (GDM; detected at 24–28 weeks of gestation in individuals without previously identified diabetes or high-risk glucose metabolism), Diabetes from other causes (e.g., monogenic diabetes syndromes, diseases of the exocrine pancreas, and drug- or chemical-induced d

In [ ]:
split_length = 0
for o in opt:
  print(len(o))
  split_length += len(o)
print(split_length)

6
6
6
7
3
6
6
3
4
4
5
7
5
7
75


In [ ]:
len(opt)


14

In [ ]:
from sentence_transformers import SentenceTransformer

#define the model you want to use
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [ ]:
from typing import List, Tuple
from tqdm import tqdm
import torch

def encode_content_splits(content_splits: List[List[str]],
                          model: SentenceTransformer,
                          device: str = 'cuda:0' if torch.cuda.is_available() else 'cpu'
                          ) -> List[List[Tuple[str, List[float]]]]:
    """
    Encode content splits as vector embeddings from a vector of content splits,
    where each vector of splits is a single podcast episode.

    Example
    -------
    content_splits = [['There is so much good content on this show.', 'This would normally be a really long block of content.'],
                      ['The content here is really good as well.', 'More content, blah, blah.']]

    output = encode_content_splits(content_splits, model)

    output >>> [
        EPISODE 1 -> [('There is so much good content on this show.', [0.01780361, -0.01932660, 0.00361164, -0.05896509, 0.01915103, 0.01608088, 0.11361098, 0.03599481, -0.01730668, -0.03303481, -0.10089817, 0.02348473]),
                      ('This would normally be a really long block of content.', [...]),
                      ...],
        EPISODE 2 -> [('The content here is really good as well.', [...]),
                      ('More content, blah, blah.', [...]),
                      ...],
        ...
    ]
    """
    text_vector_tuples = []

    model.to(device)

    for episode_splits in tqdm(content_splits, desc="Encoding content splits"):
        with torch.no_grad():
            episode_embeddings = model.encode(episode_splits, convert_to_tensor=True, device=device)
            episode_embeddings = episode_embeddings.cpu().tolist()
        text_vector_tuples.append(list(zip(episode_splits, episode_embeddings)))


    return text_vector_tuples

In [ ]:
#define the model you want to use
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

text_vector = encode_content_splits(opt, model)
text_vector

Encoding content splits: 100%|██████████| 14/14 [00:11<00:00,  1.20it/s]


[[('Diagnostic Tests for Diabetes: There is insufficient evidence to support the use of continuous glucose monitoring for screening or diagnosing prediabetes or diabetes. In the absence of unequivocal hyperglycemia (e.g., hyperglycemic crisis), diagnosis of type 2 diabetes requires confirmatory testing, which can be a different test on the same day or the same test on a different day. Marked discordance between A1C and repeated blood glucose measurements should raise the possibility of a problem or interference with either test. Classification: Classification of diabetes type is not always straightforward at presentation, and misdiagnosis is common. Section 2: Diagnosis and Classification of Diabetes: Gestational diabetes mellitus (GDM; detected at 24–28 weeks of gestation in individuals without previously identified diabetes or high-risk glucose metabolism), Diabetes from other causes (e.g., monogenic diabetes syndromes, diseases of the exocrine pancreas, and drug- or chemical-induced

In [ ]:
def join_metadata(corpus: list[dict],
                  text_vector_list: list[list[tuple[str, list]]],
                  unique_id_field: str = 'doc_id',
                  content_field: str = 'content',
                  embedding_field: str = 'content_embedding'
                  ) -> list[dict]:
    """
    Combine episode metadata from the original corpus with their associated text/vector tuples.
    Creates a new dictionary for each text/vector combination.

    Args:
        corpus (list[dict]): A list of dictionaries representing the original corpus, where each
            dictionary contains metadata for a single episode.
        text_vector_list (list[list[tuple[str, list]]]): A list of lists, where each inner list
            contains tuples of text splits and their corresponding vector embeddings for a single episode.
        unique_id_field (str, optional): The name of the field in the corpus dictionary that contains
            the unique identifier for each episode. Defaults to 'video_id'.
        content_field (str, optional): The name of the field to use for the text content in the output
            dictionaries. Defaults to 'content'.
        embedding_field (str, optional): The name of the field to use for the vector embeddings in the
            output dictionaries. Defaults to 'content_embedding'.

    Returns:
        list[dict]: A list of dictionaries, where each dictionary contains the metadata for a single
            text split, including the original metadata fields, a unique identifier field, a content
            field with the text split, and an embedding field with the vector embedding.
    """
    joined_documents = []

    for episode_meta, episode_tuples in zip(corpus, text_vector_list):
        unique_id = episode_meta[unique_id_field]
        for i, (text, vector) in enumerate(episode_tuples):
            new_dict = episode_meta.copy()
            doc_id = f"{unique_id}_{i}"
            new_dict.update({
                'chunk_id': doc_id,
                content_field: text,
                embedding_field: vector
            })
            joined_documents.append(new_dict)

    return joined_documents

In [ ]:
#create your dataset when ready
joined_corpus = join_metadata(res, text_vector)

In [ ]:
print(joined_corpus[0])

{'doc_id': 'd_1', 'content': 'Diagnostic Tests for Diabetes: There is insufficient evidence to support the use of continuous glucose monitoring for screening or diagnosing prediabetes or diabetes. In the absence of unequivocal hyperglycemia (e.g., hyperglycemic crisis), diagnosis of type 2 diabetes requires confirmatory testing, which can be a different test on the same day or the same test on a different day. Marked discordance between A1C and repeated blood glucose measurements should raise the possibility of a problem or interference with either test. Classification: Classification of diabetes type is not always straightforward at presentation, and misdiagnosis is common. Section 2: Diagnosis and Classification of Diabetes: Gestational diabetes mellitus (GDM; detected at 24–28 weeks of gestation in individuals without previously identified diabetes or high-risk glucose metabolism), Diabetes from other causes (e.g., monogenic diabetes syndromes, diseases of the exocrine pancreas, and

In [ ]:
import json

# Pretty-print the JSON object
pretty_data = json.dumps(joined_corpus[6], indent=4)
print(pretty_data)


{
    "doc_id": "d_2",
    "content": "The document titled Prevention or Delay of Diabetes abridged from DIABETESJOURNALS.ORG/CLINICAL provides comprehensive guidance on screening for Type 2 Diabetes and associated care strategies. It begins with diet and physical activity recommendations for adults at risk, emphasizing whole grains, legumes, nuts, fruits, and vegetables while minimizing refined and processed foods. Healthy eating patterns suggested include Mediterranean-style, low-carbohydrate, vegetarian or plant-based, and DASH (Dietary Approaches to Stop Hypertension). Physical activity recommendations include at least 150 minutes per week of moderate-intensity activity, such as brisk walking, potentially including resistance or strength training, and breaking up prolonged sedentary time. The document outlines a three-step approach to screening: why, how, and when. Lab testing is described as safe and cost-effective, providing an opportunity to address cardiovascular risk factors s

In [ ]:
print(len(joined_corpus))

75


In [ ]:
!pip install chromadb

In [ ]:
import chromadb
client = chromadb.PersistentClient(path="../database/chroma_db")

In [ ]:
collection = client.create_collection(name="healthtech_database")

In [ ]:
collection = client.get_collection(name="healthtech_database")

In [ ]:
collection

In [ ]:
client.heartbeat()

1719750741143284108

In [ ]:
list_embeddings=[item["content_embedding"] for item in joined_corpus]
print(len(list_embeddings))

75


In [ ]:
list_ids=[item["chunk_id"] for item in joined_corpus]
print(len(list_embeddings))

75


In [ ]:
list_docs=[item["content"] for item in joined_corpus]
print(len(list_embeddings))

75


In [ ]:
list_metadatas=[]

for item in joined_corpus:
  new_dict={
        "doc_id":item["doc_id"],
        "summary":item["summary"]
    }
  list_metadatas.append(new_dict)

print(len(list_metadatas))


75


In [ ]:
collection.add(
    documents=list_docs,
    embeddings=list_embeddings,
    metadatas=list_metadatas,
    ids=list_ids
)

In [ ]:
results = collection.query(
    query_texts=["What are the specific recommendations for screening and treatment of nonalcoholic fatty liver disease (NAFLD) in people with type 2 diabetes who are over 50 years old?"], # Chroma will embed this for you
    n_results=5 # how many results to return
)
print(results)

{'ids': [['d_3_2', 'd_13_1', 'd_13_2', 'd_14_4', 'd_3_3']], 'distances': [[0.6673091891166761, 0.7141609539341254, 0.9381126989067514, 0.9543030944286176, 0.9590638322007644]], 'metadatas': [[{'doc_id': 'd_3', 'summary': "The comprehensive medical evaluation and assessment of comorbidities in diabetes care focus on preventing complications and optimizing the quality of life. This involves a collaborative approach with an interprofessional team, regular follow-ups, and active patient engagement. Key components include screening for autoimmune conditions like thyroid disease and other related diseases in type 1 diabetes, and addressing higher fracture risks associated with diabetes by recommending bone health optimization strategies, including regular dual-energy X-ray absorptiometry and appropriate medications.Diabetes also increases the risk of certain cancers, but screening recommendations remain the same as for the general population. Nonalcoholic fatty liver disease (NAFLD) is preva

In [ ]:
import json

# Pretty-print the JSON object
pretty_data = json.dumps(results, indent=4)
print(pretty_data)

{
    "ids": [
        [
            "d_3_2",
            "d_13_1",
            "d_13_2",
            "d_14_4",
            "d_3_3"
        ]
    ],
    "distances": [
        [
            0.6673091891166761,
            0.7141609539341254,
            0.9381126989067514,
            0.9543030944286176,
            0.9590638322007644
        ]
    ],
    "metadatas": [
        [
            {
                "doc_id": "d_3",
                "summary": "The comprehensive medical evaluation and assessment of comorbidities in diabetes care focus on preventing complications and optimizing the quality of life. This involves a collaborative approach with an interprofessional team, regular follow-ups, and active patient engagement. Key components include screening for autoimmune conditions like thyroid disease and other related diseases in type 1 diabetes, and addressing higher fracture risks associated with diabetes by recommending bone health optimization strategies, including regular dual

In [ ]:
from google.colab import drive
drive.mount('/content/drive')